<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/main/notebooks/01_Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Dataset Generation
This chapter focuses on creating and expanding the dataset. It covers data collection, preprocessing and formatting to ensure compatibility with the model. The goal is to generate high-quality input data that improves model performance.

The secret `GOOGLE_API_KEY` must be configured in your Google Colab environment for proper execution.

**Note**: The 503 error that occasionally occurs when calling the Gemini API is due to temporary issues on Google's side, such as server overload. This happens when the API service is unable to process incoming requests at that moment. A similar issue has been reported in the Google AI forum: https://discuss.ai.google.dev/t/error-the-model-is-overloaded/48410.

## Imports and Setup

In [1]:
import ast
import random
import time
from datetime import datetime, timedelta
from itertools import combinations

import pandas as pd
import google.generativeai as genai
from google.colab import userdata


# API Setup
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash-exp')


def generate_text(prompt):
    """
    Generates text based on the provided prompt using the Gemini API. The function sends the prompt
    to the model, with a generation configuration that includes a temperature of 2.0 for creative output.
    It then waits for 5 seconds to avoid exceeding API limits before returning the generated text.
    """
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.GenerationConfig(
                temperature=2.0, # creative output
            )
        )
        time.sleep(10) # avoid exceeding API limits
        return response.text.strip()
    except Exception as e:
        exit("Error during API call: ", e)

**Gemini Rate Limits**

Although Gemini 2.0 Flash is significantly faster than its predecessor, it comes with a key limitation. According to the rate limits (https://ai.google.dev/gemini-api/docs/models/gemini#rate-limits), the 2.0 version allows only 10 requests per minute (RPM), compared to 15 RPM in version 1.5. Despite this, we opted for the latest version and implemented a timeout (significantly longer than necessary to account for various issues) to ensure we do not exceed the limit.  

One advantage of Gemini 2.0 Flash, however, is its significantly higher token processing capacity. It can generate up to 4 million tokens per minute (TPM), compared to just 1 million TPM in version 1.5, making it much better suited for handling longer responses.  

While discussing rate limits, it's also worth noting a third restriction: the Requests per Day (RPD) limit. Both models share the same cap of 1,500 RPD, which must be considered when scaling up request volumes.

## Load the data from the provided questionnaires

In [2]:
dfs = list()

for q in range(1, 6):
    url = f'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/questionnaires/questionnaire{q}.json'
    temp_df = pd.read_json(url)

    # Unpack options into an array
    temp_df['options'] = temp_df['options'].apply(lambda x: [option['option'] for option in x])

    # Remove options for specific question types
    temp_df.loc[temp_df['type'].isin(['TEXT', 'NUMBER', 'DATE']), 'options'] = None

    dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)

df.head()

,id,type,question,options
0,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,SINGLE_SELECT,Data processing consent,"[Yes, No]"
1,12e1ed1d-edaa-4e93-8645-de3850e998f9,SINGLE_SELECT,Customer group,"[End User, Wholesaler, Distributor, Consultant..."
2,625012ae-9192-4cf6-a73d-55e1813d6014,MULTI_SELECT,Products interested in,"[MY-SYSTEM, Notion, JTS, JS EcoLine, AKW100, A..."
3,0699fc5a-34a4-4160-bda1-fb135a3615da,MULTI_SELECT,What kind of follow up is planned,"[Email, Phone, Schedule a Visit, No action]"
4,815dab84-bc5e-4764-9777-0c0126e3173e,MULTI_SELECT,Who to copy in follow up,"[Stephan Maier, Joachim Wagner, Erik Schneider..."


## Generate Additional Questions
In this chapter, a set number of new questions is generated for each question type using various prompts. The process leverages the Gemini API 2.0 (experimental), as it provides the best results and fastest performance. This approach ensures a diverse and well-balanced dataset expansion.

In [3]:
# Prompts
select_question = f"""
    Generate a unique question that could be asked to an app user in a business
    context, designed as either a single-choice or multiple-choice question.
    Provide the question and an array of answer options in the format:
    [question, [option1, option2, ..., optionN]]
    Respond strictly in this format without additional explanations, comments,
    or text.
"""

text_question = f"""
    Generate a unique question that could be asked to an app user in a business
    context, designed as an open text entry question. Return the generated
    question without additional explanations, comments, or text.
"""

date_question = f"""
    Generate a unique question asking an app user in a business context to
    provide a date reference in the future. Return the generated question
    without additional explanations, comments, or text.
"""

number_question = f"""
    Generate a unique question asking an app user in a business context to
    provide a phone number. Return the generated question without additional
    explanations, comments, or text.
"""

In [4]:
add_data = list()
n_questions_per_type = 10

for t in df['type'].unique():
    for _ in range(n_questions_per_type):
        if t == "SINGLE_SELECT":
            question, options = ast.literal_eval(generate_text(select_question))
        elif t == "MULTI_SELECT":
            question, options = ast.literal_eval(generate_text(select_question))
        elif t == "TEXT":
            question, options = generate_text(text_question), None
        elif t == "DATE":
            question, options = generate_text(date_question), None
        elif t == "NUMBER":
            question, options = generate_text(number_question), None
        else:
            exit(f"Unhandled question type: {t}")

        add_data.append({"type": t, "question": question, "options": options})

    time.sleep(30)
    print(f"Generated {n_questions_per_type} questions of type: {t}")

add_df = pd.DataFrame(add_data)

add_df.head()

Generated 10 questions of type: SINGLE_SELECT
Generated 10 questions of type: MULTI_SELECT
Generated 10 questions of type: TEXT
Generated 10 questions of type: DATE
Generated 10 questions of type: NUMBER


,type,question,options
0,SINGLE_SELECT,"Regarding your recent customer interaction, wh...","[Response time, Information accuracy, Professi..."
1,SINGLE_SELECT,"Considering the last project, which communicat...","[Email, Instant Messaging (e.g., Slack), Video..."
2,SINGLE_SELECT,"Regarding your primary team's workflow, which ...","[Internal communication, Resource allocation, ..."
3,SINGLE_SELECT,"Considering your project goals, which of the f...","[Revenue Growth, Cost Optimization, Market Pen..."
4,SINGLE_SELECT,Thinking about your recent project contributio...,"[Technical coding, Strategic planning, Communi..."


## Process Questions
This chapter focuses on generating spoken-style answers based on the question type. A distinction is made between different question types, including single-select, multi-select, text, number, and date. Each type is processed using a dedicated handler to ensure appropriate answer generation.

In [5]:
def process_question(data):
    """
    Generate spoken-style answers for the passed question.
    A distinction is made between the different types of questions.
    """
    type_handlers = {
        "SINGLE_SELECT": handle_single_select,
        "MULTI_SELECT": handle_multi_select,
        "TEXT": handle_text,
        "NUMBER": handle_number,
        "DATE": handle_date,
    }

    data_type = data.get('type')
    handler = type_handlers.get(data_type)

    time.sleep(10)

    if handler:
        return handler(data)
    else:
        exit(f"Unhandled data type: {data_type}")

### Single-Select

In [6]:
def generate_single_answers(question, option):
    prompt = f"""
        You are an app user responding to the following question in a
        conversational, spoken style. You enjoy talking, so you respond with
        full sentences rather than a simple 'yes' or 'no'.
        Question: '{question}'
        Your response must explicitly convey the provided content: '{option}'.
        Generate 5 unique and varied responses, formatted as:
        'answer1§answer2§...§answer5'.
        Return only the generated responses in the specified format, without
        any additional explanation or comments.
        Do not use quotation marks in the response.
    """
    return generate_text(prompt)


def handle_single_select(data):
    """
    Example output:
    intended_answer: ['Yes', 'Yes', ..., 'No', 'No', ...]
    context: ['Yeah, sure thing, ...', 'Nope, I'd rather ...', ...]
    """
    intended_answer = list()
    context = list()

    for option in data['options']:
        response_text = generate_single_answers(data['question'], option)
        texts_array = [answer.strip() for answer in response_text.split("§")]

        intended_answer.extend([option] * 5)
        context.extend(texts_array)

    print(f"Generated context for question: '{data['question']}'")
    return intended_answer, context

### Multi-Select

In [7]:
def generate_multi_answers(question, options):
    options_text = ", ".join(options)
    prompt = f"""
        You are an app user responding to the following question in a
        conversational, spoken style. You enjoy talking, so you respond with
        full sentences rather than a simple 'yes' or 'no'.
        Question: '{question}'
        Your response must contain all of the following text elements
        explicitly to be valid: '{options_text}'.
        Generate 5 unique and varied responses, formatted as:
        'answer1§answer2§...§answer5'.
        Return only the generated responses in the specified format, without
        any additional explanation or comments.
        Do not use quotation marks in the response.
    """
    return generate_text(prompt)


def handle_multi_select(data):
    """
    Example output:
    intended_answer: [["MY-SYSTEM", "Notion"], ["Notion"], ...]
    context: ['Yeah, that would be MY-SYSTEM and Notion, ...',
        'Hmm, I think I'm mainly interested in Notion ...', ...]
    """
    intended_answer = list()
    context = list()

    # Generate all possible combinations of options (subsets)
    all_combinations = list()
    for r in range(1, len(data['options']) + 1):
        all_combinations.extend(list(combinations(data['options'], r)))

    # Shuffle combinations for randomness
    random.shuffle(all_combinations)

    # Only generate answers for a random sample of combinations
    selected_combinations = random.sample(all_combinations, min(5, len(all_combinations)))

    for combo in selected_combinations:
        response_text = generate_multi_answers(data['question'], combo)
        texts_array = [answer.strip() for answer in response_text.split("§")]

        intended_answer.extend([combo] * 5)
        context.extend(texts_array)

    print(f"Generated context for question: '{data['question']}'")
    return intended_answer, context

### Text

In [8]:
def generate_text_answers(question):
    prompt = f"""
        You are an app user responding to the following question in a
        conversational, spoken style. You enjoy talking, so you respond with
        full sentences rather than a simple 'yes' or 'no'.
        Question: '{question}'
        Generate 5 unique and varied responses, formatted as:
        'answer1§answer2§...§answer5'.
        Return only the generated responses in the specified format, without
        any additional explanation or comments.
        Do not use quotation marks in the response.
    """
    return generate_text(prompt)


def handle_text(data):
    """
    Example output:
    intended_answer: [None, None, ...]
    context: ['You can only reach me on ...', 'I have no notes to add.', ...]
    """
    intended_answer = list()
    context = list()

    response_text = generate_text_answers(data['question'])
    texts_array = [answer.strip() for answer in response_text.split("§")]

    intended_answer.extend([None] * 5)
    context.extend(texts_array)

    print(f"Generated context for question: '{data['question']}'")
    return intended_answer, context

### Number

In [9]:
def generate_phone_number():
    """
    Generates a random phone number in an international format.
    """
    country_code = random.choice(["+1", "+44", "+49", "+33"])
    area_code = random.randint(100, 999)
    local_number = f"{random.randint(100, 999)}-{random.randint(1000, 9999)}"
    return f"{country_code}-{area_code}-{local_number}"


def generate_number_answers(question, option):
    prompt = f"""
        You are an app user responding to the following question in a
        conversational, spoken style. You enjoy talking, so you respond with
        full sentences rather than a simple 'yes' or 'no'.
        Question: '{question}'
        Your response must contain the following phone number explicitly
        to be valid: '{option}'.
        Generate 5 unique and varied responses, formatted as:
        'answer1§answer2§...§answer5'.
        Return only the generated responses in the specified format, without
        any additional explanation or comments.
        Do not use quotation marks in the response.
    """
    return generate_text(prompt)


def handle_number(data):
    """
    Example output:
    intended_answer: ['+44-7700-900123', ...]
    context: ['My number is +44-7700-900123.', ...]
    """
    intended_answer = list()
    context = list()

    phone_numbers = [generate_phone_number() for _ in range(5)]

    for option in phone_numbers:
        response_text = generate_number_answers(data['question'], option)
        texts_array = [answer.strip() for answer in response_text.split("§")]

        intended_answer.extend([option] * 5)
        context.extend(texts_array)

    print(f"Generated context for question: '{data['question']}'")
    return intended_answer, context

### Date

In [10]:
def generate_date_answers(question):
    prompt = f"""
        You are an app user responding to the following question in a
        conversational, spoken style. You enjoy talking, so you respond with
        full sentences rather than a simple 'yes' or 'no'.
        Question: '{question}'
        Your answer must contain a time reference in the future, such as
        'tomorrow', 'in three weeks', etc.
        Build a natural-sounding spoken response around this time reference.
        Generate 5 unique and varied responses, each incorporating a different
        future time reference, formatted as:
        'answer1§answer2§...§answer5§reference1§reference2§...§reference5'.
        Return only the generated responses in the specified format, without
        any additional explanation or comments.
        Do not use quotation marks in the response.
    """
    return generate_text(prompt)


def handle_date(data):
    """
    Example output:
    intended_answer: ['tomorrow', 'in three weeks', ...]
    context: ['Tomorrow would be good ...', 'How about in three weeks ...', ...]
    """
    response_text = generate_date_answers(data['question'])
    response_text = response_text.strip('"').strip("'")
    texts_array = [answer.strip() for answer in response_text.split("§")]

    intended_answer = texts_array[5:]
    context = texts_array[:5]

    print(f"Generated context for question: '{data['question']}'")
    return intended_answer, context

## Create User Inputs (Context)
This section focuses on generating relevant user inputs for each question to provide meaningful context. By incorporating realistic responses, the model can better understand different question types and generate more natural, conversational answers. The context ensures that each question is framed appropriately, improving the quality and coherence of the generated responses.

In [11]:
def generate_timestamp():
    """
    Generate random timestamp within the last 30 days.
    """
    start_date = datetime.now() - timedelta(days=30)
    random_seconds = random.randint(0, 30 * 24 * 60 * 60)
    random_date = start_date + timedelta(seconds=random_seconds)
    return random_date


df = pd.concat([df, add_df], ignore_index=True)

new_cols = ['intended_answer', 'context']

df[new_cols] = df.apply(
    lambda row: pd.Series(process_question(row)), axis=1
)

df['timestamp'] = [generate_timestamp() for _ in range(len(df))]

Generated context for question: 'Data processing consent'
Generated context for question: 'Customer group'
Generated context for question: 'Products interested in'
Generated context for question: 'What kind of follow up is planned'
Generated context for question: 'Who to copy in follow up'
Generated context for question: 'Would you like to receive marketing information from via e-mail?'
Generated context for question: 'What industry are you operating in?'
Generated context for question: 'What products are you interested in?'
Generated context for question: 'Notes'
Generated context for question: 'What type of company is it?'
Generated context for question: 'What is the size of your company?'
Generated context for question: 'When do you wish to receive a follow-up?'
Generated context for question: 'Any additional notes?'
Generated context for question: 'Which language is wanted for communication? '
Generated context for question: 'What is the type of contact?'
Generated context for ques

In [12]:
# df = df.explode(new_cols, ignore_index=True)

# Exploding separately is more robust, preventing errors from uneven list lengths
for col in new_cols:
    df = df.explode(col, ignore_index=True)

In [13]:
print("Number of unique questions:", df['question'].nunique())
print("Number of Q&A-pairs (= number of rows in the dataset):", len(df))

Number of unique questions: 74
Number of Q&A-pairs (= number of rows in the dataset): 34225


In [14]:
# Save dataset to a new JSON file
df.to_json('qa_dataset.json', orient='records', indent=4)